In [1]:
import os
import pandas as pd

In [3]:

data_directory = '/Users/derek/Documents/Github_Repos/graphy_financials/data'

tsv_path_to_process = []

for folder_name in os.listdir(data_directory):
    folder_path = os.path.join(data_directory, folder_name)
    
    if os.path.isdir(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file_name in files:
                if file_name.endswith('DISCLOSURE.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    
                    tsv_path_to_process.append(tsv_path)
                    

In [95]:
# tsv_2023 = [x for x in tsv_path_to_process if "2023" in x]

disclosure_dataframes = []

for tsv_file in tsv_path_to_process:
    df = pd.read_csv(tsv_file, sep="\t")
    disclosure_dataframes.append(df)

combined_disclosure_df = pd.concat(disclosure_dataframes, ignore_index=True)


# combined_disclosure_df[['ACCESSION_NUMBER','COMPENSATIONAMOUNT']]
# combined_disclosure_df['ACCESSION_NUMBER']

combined_disclosure_df.rename(columns = {
    'ACCESSION_NUMBER': 'accession_number',
    'COMPENSATIONAMOUNT': 'compensation_amount_description', 
    'FINANCIALINTEREST': 'financial_interest',
    'SECURITYOFFEREDTYPE': 'security_offered_type',
    'SECURITYOFFEREDOTHERDESC': 'security_offered_other_desc',
    'NOOFSECURITYOFFERED': 'no_of_security_offered',
    'PRICE': 'price',
    'PRICEDETERMINATIONMETHOD': 'price_determination_method', 
    'OFFERINGAMOUNT': 'offering_amount',
    'OVERSUBSCRIPTIONACCEPTED': 'oversubscription_accepted',
    'OVERSUBSCRIPTIONALLOCATIONTYPE': 'oversubscription_allocation_type',
    'DESCOVERSUBSCRIPTION': 'desc_oversubscription',
    'MAXIMUMOFFERINGAMOUNT': 'maximum_offering_amount',
    'DEADLINEDATE': 'deadline_date',
    'CURRENTEMPLOYEES': 'current_employees',
    'TOTALASSETMOSTRECENTFISCALYEAR': 'total_assets_most_recent_fiscal_year',
    'TOTALASSETPRIORFISCALYEAR': 'total_assets_prior_fiscal_year',
    'CASHEQUIMOSTRECENTFISCALYEAR': 'cash_equity_most_recent_fiscal_year',
    'CASHEQUIPRIORFISCALYEAR': 'cash_equity_prior_fiscal_year',
    'ACTRECEIVEDRECENTFISCALYEAR': 'act_received_recent_fiscal_year', 
    'ACTRECEIVEDPRIORFISCALYEAR': 'act_received_prior_fiscal_year',
    'SHORTTERMDEBTMRECENTFISCALYEAR': 'short_term_debt_recent_fiscal_year',
    'SHORTTERMDEBTPRIORFISCALYEAR': 'short_term_debt_prior_fiscal_year',
    'LONGTERMDEBTRECENTFISCALYEAR': 'long_term_debt_recent_fiscal_year',
    'LONGTERMDEBTPRIORFISCALYEAR': 'long_term_debt_prior_fiscal_year',
    'REVENUEMOSTRECENTFISCALYEAR': 'revenue_most_recent_fiscal_year',
    'REVENUEPRIORFISCALYEAR': 'revenue_prior_fiscal_year', 
    'COSTGOODSSOLDRECENTFISCALYEAR': 'cost_goods_sold_recent_fiscal_year',
    'COSTGOODSSOLDPRIORFISCALYEAR': 'cost_goods_sold_prior_fiscal_year',
    'TAXPAIDMOSTRECENTFISCALYEAR': 'tax_paid_most_recent_fiscal_year',
    'TAXPAIDPRIORFISCALYEAR': 'tax_paid_prior_fiscal_year',
    'NETINCOMEMOSTRECENTFISCALYEAR': 'net_income_most_recent_fiscal_year',
    'NETINCOMEPRIORFISCALYEAR': 'net_income_prior_fiscal_year'
}, inplace = True )

combined_disclosure_df.info(verbose=True, max_cols=None)

TypeError: info() got an unexpected keyword argument 'columns'

In [89]:
# change data type
combined_disclosure_df = combined_disclosure_df.astype({
    'accession_number': 'object',
    'compensation_amount_description': 'object',
    'financial_interest': 'object',
    'security_offered_type': 'object',
    'security_offered_other_desc': 'object',
    'no_of_security_offered': 'float',
    'price': 'float',
    'price_determination_method': 'object',
    'offering_amount': 'float',
    'oversubscription_accepted': 'object',
    'oversubscription_allocation_type': 'string',
    'desc_oversubscription': 'string',
    'maximum_offering_amount': 'string'
})

# combined_disclosure_df['accession_number'][1]

# combined_disclosure_df.info()
combined_disclosure_df['oversubscription_accepted'].to_csv('testing_oversub.csv')